In [9]:
# pip install scikit-learn
# pip install keras
# pip install tensorflow

# import os
# import sys
import pandas as pd
import numpy as np

# from sklearn.preprocessing import Normalizer, MinMaxScaler
# from sklearn.compose import ColumnTransformer
# import pandas as pd

# from tensorflow.keras.models import load_model


# import keras

# pd.set_option('display.max_columns', 100)



In [ ]:
df = pd.read_csv("prueba.csv")

df.drop(columns=['id','date_time_utc','unix_time'], inplace=True)


df.head(10)

In [ ]:
scaler = MinMaxScaler()


# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)



In [ ]:
# Convert the DataFrame to a NumPy array
raw_data = df_normalized.values
close = df_normalized['close_price'].values
close = close.reshape((len(close),1))
# Display the NumPy array
print(type(raw_data))
print(raw_data.shape)
print(close.shape)

In [ ]:
# Load the model from the .h5 file
model = load_model('lstm_model_1_ahead.h5')


In [ ]:
pred_dataset = keras.preprocessing.timeseries_dataset_from_array(
                    raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=1,
                    sequence_stride=1,
                    sequence_length=180,
                    shuffle=False, # Shouldn't the shuffle be set to 0?
                    seed=33,
                    batch_size=32,
                    start_index=num_train_samples,
                    end_index=num_train_samples + num_val_samples
                    )

predictions = model.predict(pred_dataset)

In [41]:

# Function to add incremental number starting from a specific value
def add_incremental_number(row, increment, starting_number):
    return starting_number + row.name * increment


def merge_dataframes(predicted_prices, df_initial,increment):


    starting_number = df_initial['unix_time'].max()+increment

    last_close_price = df.loc[(df_initial['unix_time'] == df_initial['unix_time'].max()), "close_price"].iloc[0]

    predicted_prices['unix_time'] = predicted_prices.apply(add_incremental_number, args=(increment, starting_number), axis=1)

    # insert the last close price as the open price of the first row
    predicted_prices['open_price'] = predicted_prices['close_price'].shift(1)


    # Find the index of the row with the specified unix_time
    index = predicted_prices.loc[predicted_prices['unix_time'] == starting_number].index[0]

    # Replace the open_price value of the specified row with 20
    predicted_prices.at[index, 'open_price'] = last_close_price

    # craete a new column with 0 values for the volume
    predicted_prices['volume'] = 0

    # Create the new 'low_price' column with the specified conditions
    predicted_prices['low_price'] = predicted_prices.apply(lambda row: row['close_price'] if row['close_price'] < row['open_price']
                                else row['open_price'], axis=1)


    # Create the new 'high_price' column with the specified conditions
    predicted_prices['high_price'] = predicted_prices.apply(lambda row: row['open_price'] if row['close_price'] < row['open_price']
                                else row['close_price'], axis=1)


    result_df = pd.concat([df_initial, predicted_prices], axis=0)
    result_df.sort_values('unix_time', ascending=True, inplace=True)

    return result_df



In [47]:
predicted_prices = pd.DataFrame({'close_price': [3232]})
increment = 3600

df2 = pd.read_csv("prueba.csv")
df_initial = df[["unix_time", "open_price", "close_price", "low_price", "high_price", "volume"]].copy()
df_initial


,unix_time,open_price,close_price,low_price,high_price,volume
0,1681603200,2090.61,2118.67,2072.72,2141.54,319880.4393
1,1681689600,2118.66,2074.00,2056.25,2120.51,426972.7071
2,1681776000,2073.99,2103.50,2051.00,2125.00,414244.6135
3,1681862400,2103.51,1933.73,1923.03,2104.60,775389.0602
4,1681948800,1933.74,1942.98,1914.43,1982.99,625336.2495


In [48]:
result_df = merge_dataframes(predicted_prices, df_initial,increment)
result_df

,unix_time,open_price,close_price,low_price,high_price,volume
0,1681603200,2090.61,2118.67,2072.72,2141.54,319880.4393
1,1681689600,2118.66,2074.00,2056.25,2120.51,426972.7071
2,1681776000,2073.99,2103.50,2051.00,2125.00,414244.6135
3,1681862400,2103.51,1933.73,1923.03,2104.60,775389.0602
4,1681948800,1933.74,1942.98,1914.43,1982.99,625336.2495
0,1681952400,1942.98,3232.00,1942.98,3232.00,0.0000
